
# **ATENÇÃO**
A lista deverá ser entregue tanto no formato .pdf e em .py ou .ipynb


# **Rotação de uma Imagem**
Uma opção seria utilizar a função de rotação do OpenCV. A ideia do exercício é não utilizar esta função, mas sim implementá-la manualmente (para compreender o que está ocorrendo nos bastidores). Para realizar este exercício, você deve preencher as lacunas da função escrita no código em anexo. Esta função tem como entrada:

.uma imagem (img),

.o ângulo de rotação (angulo),

.o centro de rotação (centro),

e devolve a imagem rotacionada.

In [1]:
import cv2 as cv
import sys
import numpy as np

In [2]:
def my_rotation(img, angulo, centro):
    
    # nessa primeira parte, vamos definir a transformação que leva a posicao dos pixels da imagem original
    # para a posicao dos pixels do imagem rotacionada.
    
    # a primeira matriz de translação muda a origem das coordenadas do canto da imagem para o centro da imagem
    matriz_translacao = np.array([[1, 0, centro[0]], [0, 1, centro[1]], [0, 0, 1]])
    # a matriz de rotacao aplica a rotacao em torno da origem
    matriz_rotacao = np.array([[np.cos(angulo), -np.sin(angulo), 0], [np.sin(angulo), np.cos(angulo), 0], [0, 0, 1]])
    # a composicao coloca todas as matrizes em uma só: aplica a translacao (muda a origem), rotaciona, volta para a origem anterior
    matriz_composicao = matriz_translacao.dot(matriz_rotacao).dot(np.linalg.inv(matriz_translacao))

    # criar imagem rotacionada em preto, com mesmas dimensões da original
    height, width = img.shape[:2]
    rotated_image = np.zeros((height,width,3), np.uint8)
    # o próximo passo é percorrer cada pixel da nova imagem e verificar qual é o pixel correspondente na imagem original
    m_comp_inv = np.linalg.inv(matriz_composicao)
    for linr in range(height):
        for colr in range(width):
            pos_rot = np.matrix([linr, colr, 1]).T
            pos_orig = m_comp_inv.dot(pos_rot)
            lin = round(pos_orig[0,0]); col = round(pos_orig[1,0])
            if (lin >=0 and lin < height) and (col >= 0 and col < width):
                #opa, é um pixel pertencente à imagem original...
                rotated_image[linr, colr] = img[int(lin), int(col)]

    return rotated_image

In [3]:
img = cv.imread('Palazzo_Farnese_Fassade.jpg')
height, width = img.shape[:2]
center = (height/2, width/2)
angle = np.pi/6
my_rotated_image =  my_rotation(img, angle, center)

In [4]:
cv.imshow('Display window', my_rotated_image)
cv.waitKey(0); cv.destroyAllWindows()

Você pode comparar seu resultado com o função do Open CV.

In [5]:
# Obter o centro da imagem
height, width = img.shape[:2]
center = (width/2, height/2)

# Definir a matriz de rotação
M = cv.getRotationMatrix2D(center, 30, 1)

# Aplicar a rotação na imagem
rotated_img = cv.warpAffine(img, M, (width, height))

cv.imshow('Display window', rotated_img)
cv.waitKey(0); cv.destroyAllWindows()

# **Transformação Projetiva de uma Imagem**
Escreva uma função que tenha uma como entrada:

. uma imagem (img),

. uma matriz de uma transformação projetiva (T),

e devolva a imagem transformada.

A ideia é semelhante ao exercício 1, mas ao invés da rotação, temos uma transformação projetiva (homografia).

Teste a sua função, faça a transformação projetiva com apenas um ponto de fuga no eixo $x$ (digamos, o ponto $(2000,0)$ em coordenadas $x,y$), da imagem do Palazzo Farnese Fassade (em anexo).

Você pode usar outra imagem de sua preferência, caso queira. Não se esqueça de trabalhar com coordenadas homogênas.

In [6]:
# Escreva seu código aqui

def my_transform(img, T):
 # Criar imagem transformada com mesmas dimensões da original
    height, width = img.shape[:2]
    transform_image = np.zeros((height, width, 3), np.uint8)

    # Calcular a matriz inversa de T
    T_inv = np.linalg.inv(T)

    # Iterar sobre cada pixel na imagem transformada
    for lin_t in range(height):
        for col_t in range(width):
            # Calcular as coordenadas originais transformadas
            coords_originais = np.dot(T_inv, [col_t, lin_t, 1])
            if coords_originais[2] != 0:
                # Normalizar as coordenadas originais
                x_orig = int(coords_originais[0] / coords_originais[2])
                y_orig = int(coords_originais[1] / coords_originais[2])

            # Verificar se as coordenadas originais estão dentro dos limites da imagem original
            if 0 <= x_orig < width and 0 <= y_orig < height:
                # Copiar o pixel da imagem original para a imagem transformada
                transform_image[lin_t, col_t] = img[y_orig, x_orig]

    return transform_image


In [7]:
img = cv.imread('Ecole_polytechnique.png')
T = np.array([[1,0,0],
              [0,1,0],
              [0.0002,0,1]])
my_tranform_img =  my_transform(img, T)

In [8]:
cv.imshow('Display window', my_tranform_img)
cv.waitKey(0); cv.destroyAllWindows()

Você pode comparar seu resultado com o função do Open CV.


In [9]:
# Definir a matriz de transformação projetiva
T = np.array([[1,0,0],
              [0,1,0],
              [0.0002,0,1]])

# Aplicar a transformação projetiva à imagem
imagem_transformada_cv = cv.warpPerspective(img, T, (img.shape[1], img.shape[0]))

# Exibir a imagem
cv.imshow('Imagem Transformada', imagem_transformada_cv)
cv.waitKey(0); cv.destroyAllWindows()

# **Estimação da Transformação Projetiva**
**Crie uma função**

Escreva uma função que tenha uma como entrada:

. Quatro pontos “fonte” no plano projetivo, possivelmente impróprios,

. Quatro pontos “destino” no plano projetivo, possivelmente impróprios.

E devolva a transformação projetiva $T: \mathbb{R}\mathbb{P}^2 \rightarrow \mathbb{R}\mathbb{P}^2$ que leva os pontos “fonte” nos pontos
“destino”.

In [10]:
# Escreva seu código aqui

def my_estimation(lst1, lst2):
    # Verificar se as listas de pontos têm o mesmo tamanho e contêm 4 pontos cada
    if len(lst1) != 4 or len(lst2) != 4:
        raise ValueError("Ambas as listas de pontos devem conter exatamente quatro pontos.")

    # Converter as listas de pontos para matrizes NumPy
    src_mat = np.array([list(point) + [1] for point in lst1]).T
    dest_mat = np.array([list(point) + [1] for point in lst2]).T

    # Calcular a matriz de transformação projetiva usando a solução de mínimos quadrados
    estimation_T = np.linalg.lstsq(src_mat.T, dest_mat.T, rcond=None)[0].T

    return estimation_T

Encontre a transformação projetiva que leva o campo de futebol da imagem
do gol no jogo Vasco contra Flamengo (em anexo) em um retângulo correspon-
dente ao campo de futebol visto de cima em projeção ortogonal (em anexo, mas
fique a vontade para usar outra). Para isso, utilize a sua função my_estimation.

Use esta projeção para desenhar o campo
de futebol visto de cima junto com a posição do Leo Pelé, do goleiro e de um
jogador do Flamengo. Para isso, utilize a sua função my_transform.

In [15]:
# Escreva seu código aqui
# Carregar as imagens do jogo Vasco x Flamengo e do campo de futebol visto de cima
imagem_vasco_flamengo = cv.imread('Vasco_Flamengo.jpg')
imagem_campo_futebol = cv.imread("Campo_Futebol.webp")

# Definir os pontos "fonte" e "destino" para estimar a transformação projetiva
pontos_fonte = [(3, 340), (91, 211), (205, 339), (614, 146)]  # Pontos no jogo Vasco x Flamengo
pontos_destino = [(42, 258), (45, 157), (72, 257), (134, 93)]  # Pontos correspondentes no campo de futebol

# Estimar a transformação projetiva a partir dos pontos "fonte" e "destino"
matriz_transformacao = my_estimation(pontos_fonte, pontos_destino)

# Definir as coordenadas homogêneas dos pontos do goleiro e jogador no jogo Vasco x Flamengo
ponto_goleiro = (250, 300, 1) 
ponto_jogador = (550, 350, 1) 

# Aplicar a transformação nos pontos do goleiro e jogador
ponto_goleiro_transformado = np.dot(matriz_transformacao, ponto_goleiro)
ponto_jogador_transformado = np.dot(matriz_transformacao, ponto_jogador)

# Converter as coordenadas homogêneas para coordenadas cartesianas
ponto_goleiro_transformado = (int(ponto_goleiro_transformado[0] / ponto_goleiro_transformado[2]), int(ponto_goleiro_transformado[1] / ponto_goleiro_transformado[2]))
ponto_jogador_transformado = (int(ponto_jogador_transformado[0] / ponto_jogador_transformado[2]), int(ponto_jogador_transformado[1] / ponto_jogador_transformado[2]))

# Desenhar círculos nos pontos transformados na imagem do campo de futebol visto de cima
cv.circle(imagem_campo_futebol, ponto_goleiro_transformado, 5, (0, 0, 255), -1)
cv.circle(imagem_campo_futebol, ponto_jogador_transformado, 5, (0, 0, 255), -1)

# Exibir a imagem com os pontos marcados
cv.imshow('Campo de Futebol - Vista de Cima', imagem_campo_futebol)
cv.waitKey(0); cv.destroyAllWindows()

Compare seu resultado com o implementado no OpenCV (cv.warpAffine)

# **EXTRA**
# You are Fake News ... or maybe not.
Muitas vezes, os torcedores apaixonados pelo time desconfiam do resultado do VAR e das decisões de impedimento, como foi o caso do vídeo. Teste sua função na imagem da partida entre Union e Boca Juniors (em anexo) e verifique se o método mostrado no vídeo (https://www.instagram.com/papodeboleiros/reel/CvAhQazvuhu/) produz o mesmo resultado de impedimento que a função implementada (basta fazer a transformação na imagem com as "retas paralelas" do vídeo e verificar se de fato são paralelas). Comente sobre o resultado da comparação. Você acredita que este método seja sempre correto? Caso contrário, forneça um contraexemplo onde ele falhe (se possível, com uma imagem).


In [12]:
# Escreva seu código aqui

# My VAR-lidation
Utilizando suas funções my\_estimation e my\_transform, faça uma função que tem como entrada:

. Duas imagens, uma do jogo e uma do campo (img1,img2)

. Uma lista de pontos "fonte" e "destino"(lst1,lst2)

. Os pixels  relativos aos dois jogadores que estão sendo analisados (p1,p2)

E devolve a imagem do campo com uma linha marcada de ambos os jogadores, semelhante com as análises do VAR.

Perceba que basta desenhar uma linha (pintando todos os pixels com a mesma coordenada em x da mesma cor) quando a imagem do jogo estiver em projeção ortogonal e depois usar a inversa da matriz de projeção para voltar ao formato original.

Observação: a imagem provavelmente sofrerá perda de qualidade neste processo. Se desejar manter a qualidade, você poderá realizar esse processo apenas em uma imagem com a linha inserida e adicioná-la depois de transformada na imagem original.

In [13]:
# Escreva seu código aqui